# Phase 5: Business Layer

Translate forecasts into inventory reorder points, safety stock, and stockout reduction simulation.

In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from src.data_loader import REPO_ROOT, load_and_merge_data
from src.feature_engineering import build_ml_features
from src.models.ml_models import fit_lightgbm
from src.models.baselines import NaiveForecast
from src.business.inventory import (
    safety_stock,
    compute_reorder_point,
    simulate_inventory_policy,
    estimate_stockout_reduction,
    revenue_impact_estimation,
)

## 1. Generate Forecasts (ML vs Naive)

In [ ]:
data_dir = REPO_ROOT / "data" / "store-sales-time-series-forecasting"
df = load_and_merge_data(data_dir)
daily = df.groupby('date').agg({'sales': 'sum', 'onpromotion': 'sum', 'is_holiday': 'first'}).reset_index()
daily['is_holiday'] = daily['is_holiday'].fillna(False).astype(int)

feat_df = build_ml_features(daily, lags=[1, 7, 14, 30], rolling_windows=[7, 14, 30]).dropna()
feature_cols = [c for c in feat_df.columns if c not in ['date', 'sales']]

train_end = int(len(feat_df) * 0.85)
X_train = feat_df[feature_cols].iloc[:train_end].values
y_train = feat_df['sales'].iloc[:train_end].values
X_val = feat_df[feature_cols].iloc[train_end:].values
y_val = feat_df['sales'].iloc[train_end:].values

lgb_model = fit_lightgbm(X_train, y_train, X_val, y_val)
y_ml = lgb_model.predict(X_val)

naive = NaiveForecast().fit(y_train)
y_naive = naive.predict(len(y_val))

actuals = y_val
forecasts_ml = y_ml
forecasts_naive = y_naive

## 2. Reorder Point & Safety Stock

In [ ]:
lead_time_days = 7
service_level = 0.95
demand_std = np.std(y_train)
forecast_daily_mean = np.mean(y_train)

ss = safety_stock(lead_time_days, demand_std, service_level)
rop = compute_reorder_point(forecast_daily_mean, lead_time_days, demand_std, service_level)
reorder_qty = rop  # Simple: order up to ROP

print(f'Safety stock: {ss:.0f}')
print(f'Reorder point: {rop:.0f}')
print(f'Reorder quantity: {reorder_qty:.0f}')

## 3. Inventory Simulation

In [ ]:
initial_stock = rop * 1.5  # Start with buffer

# Baseline: use naive forecast to set ROP
rop_naive = np.mean(forecasts_naive) * lead_time_days + ss
res_naive = simulate_inventory_policy(actuals, initial_stock, lead_time_days, rop_naive, rop_naive)

# Optimized: use ML forecast for ROP
rop_ml = np.mean(forecasts_ml) * lead_time_days + ss
res_ml = simulate_inventory_policy(actuals, initial_stock, lead_time_days, rop_ml, rop_ml)

print('Baseline (Naive):', res_naive)
print('Optimized (ML):', res_ml)

## 4. Stockout Reduction

In [ ]:
reduction = estimate_stockout_reduction(res_naive, res_ml)
print(f"Stockout reduction: {reduction['stockout_reduction_pct']:.1f}%")
print(f"Baseline stockout days: {reduction['baseline_stockouts']}")
print(f"Optimized stockout days: {reduction['optimized_stockouts']}")

## 5. Revenue Impact

In [ ]:
rev_naive = revenue_impact_estimation(forecasts_naive, actuals)
rev_ml = revenue_impact_estimation(forecasts_ml, actuals)
print('Naive MAE:', rev_naive['mae'])
print('ML MAE:', rev_ml['mae'])
print('Improvement:', (1 - rev_ml['mae']/rev_naive['mae'])*100, '%')

## 6. Stock Levels Over Time

In [ ]:
val_dates = feat_df['date'].iloc[train_end:].values
fig, ax = plt.subplots(figsize=(12, 4))
ax.plot(val_dates, res_naive['stock_levels'], label='Baseline (Naive)', alpha=0.8)
ax.plot(val_dates, res_ml['stock_levels'], label='Optimized (ML)', alpha=0.8)
ax.axhline(rop, color='gray', linestyle='--', label='Reorder point')
ax.legend()
ax.set_title('Simulated Inventory Levels')
ax.set_ylabel('Stock')
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()